Antes de empezar: un poco de documentación para enterder ligeramente cómo funciona HTML:

https://www.w3schools.com/html/html_intro.asp

# Web Scraping: Selenium

En este notebook veremos cómo automatizar la extracción de datos de una página web simulando la navegación. Para ello utilizaremos la librería Selenium:
https://selenium-python.readthedocs.io/

El software libre Selenium es un **framework** para realizar test automatizados de software a aplicaciones web. En principio, fue desarrollado para poner a prueba páginas y apps web, pero el **WebDriver** de Selenium también puede usarse con Python para realizar scraping. Si bien Selenium en sí no está escrito en Python, con este lenguaje de programación es posible acceder a las funciones del software.

A diferencia de Scrapy y de BeautifulSoup, Selenium no trabaja con el texto fuente en HTML de la web en cuestión, sino que carga la página en un navegador sin interfaz de usuario. El navegador interpreta entonces el código fuente de la página y crea, a partir de él, un **Document Object Model** (modelo de objetos de documento o DOM). Esta interfaz estandarizada permite poner a prueba las interacciones de los usuarios. De esta forma se consigue, por ejemplo, simular clics y rellenar formularios automáticamente. Los cambios en la web que resultan de dichas acciones se reflejan en el DOM. La estructura del proceso de web scraping con Selenium es la siguiente:

URL → Solicitud HTTP → HTML → Selenium → DOM

In [2]:
# Imports
import pandas as pd
from selenium import webdriver


Deberemos descargar un archivo Chrome driver para selenium en https://chromedriver.chromium.org/downloads, es importante que compruebes la versión de Chrome que utilizas y descargaues el driver correspondiente.
Una vez descargado crearemos una variable con la dirección completa del path al chromedriver 

El objeto driver es con el que trabajaremos a partir de ahora

In [3]:
# especificamos el path hasta nuestro driver recién descargado:
chrome_driver_path = "chromedriver.exe"
options = webdriver.ChromeOptions()
#options.add_argument("headless")
# headless nos permite no tener que ver la navegación en la ventana de Chrome

In [7]:
# Creamos el driver con el que nos vamos a manejar en la sesión de scrapeo:
driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)

In [8]:
# indicamos la URL de la página web a la que queremos acceder:
url = "https://www.filmaffinity.com/es/main.html"

# el objeto driver nos va a permitir alterar el estado de la página
driver.get(url)


¡Vaya! Nos hemos encontrado con un pop-up que nos pide aceptar cookies o algo po el estilo, si nos lo queremos saltar, para poder automatizar el proceso deberemos:

1. Buscamos el botón
2. Hacemos click en el botón

Selenium nos permite buscar elementos por etiqueteas de html: https://selenium-python.readthedocs.io/locating-elements.html

CUIDADO! Los metodos de busqueda de elementos estan "duplicados". Tenemos *find_element_by_tag_name* y *find_elements_by_tag_name* el primero nos devolverá el primer elemento que encuentre (aunque haya más), el segundo nos devolverá una lista con todos los elementos que encuentre (aunque solo sea uno)

In [12]:
# Navegando en filmaffinity, en el pop-up:
elements_by_tag = driver.find_elements_by_tag_name ("button")
elements_by_class_name = driver.find_elements_by_class_name("css-47sehv")
element_by_xpath = driver.find_element_by_xpath("/html/body/div[1]/div/div/div/div[2]/div/button[2]")

Una vez tenemos los elementos podemos hacer varias cosas con ellos

Podemos extraer todos los atributos que tenga

In [15]:
# obtenemos todos sus métodos y atributos:
dir(elements_by_tag)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [ ]:
element_by_xpath.size

Podemos evaluar que tipo de elemento es (tag)

Podemos sacar el valor que tiene (el texto)

In [20]:
element_by_xpath.text


'ACEPTO'

Incluso podemos guardar una imagen del elemento

In [21]:
# Vemos que es tipo 'WebElement' y en la documentación podremos encontrar sus métodos
type(element_by_xpath)

selenium.webdriver.remote.webelement.WebElement

In [22]:
# guardamos como 'mi_imagen.png' la imagen asociada al xpath
element_by_xpath.screenshot("mi_imagen.png")

True

Evaluamos que elementos hemos encontrado por el tag:

In [23]:
for index, element in enumerate(elements_by_tag):
    print("Elemento:", index)
    print("Texto del elemento:", index, "es", element.text)
    print("El tag del elemento", index, "es", element.tag_name)
    element.screenshot("mi_imagen"+str(index)+".png")

Elemento: 0
Texto del elemento: 0 es socios
El tag del elemento 0 es button
Elemento: 1
Texto del elemento: 1 es MÁS OPCIONES
El tag del elemento 1 es button
Elemento: 2
Texto del elemento: 2 es ACEPTO
El tag del elemento 2 es button


Nos quedamos con el botón que nos interesa

In [24]:
boton_aceptar = elements_by_tag[2]
boton_aceptar


<selenium.webdriver.remote.webelement.WebElement (session="b8a3158143a469d27d6821d2b81a648f", element="f22ff5c1-1626-421b-9b9f-51461ed314fc")>

Si el elemento es interactivo podremos hacer más cosas además de las anteriores. Por ejemplo: hacer click

In [26]:
boton_aceptar.click()

Buscamos una película por título

In [25]:
from selenium.webdriver.common.keys import Keys


In [27]:
buscador = driver.find_element_by_xpath("/html/body/div[2]/div[1]/div/div[2]/form/input[1]")

In [ ]:
buscador.send_keys("Interestellar")

In [28]:
# una vez escrita la búsqueda deberíamos poder activarla:
buscador.send_keys(Keys.ENTER)

In [29]:
# volvemos a la página anterior
driver.back()

### Vamos a buscar todas las películas que se estrenan el próximo viernes

1. Cogemos los containers que hay en la zona central

In [30]:
menu_lateral = driver.find_element_by_id("lsmenu")
menu_lateral

<selenium.webdriver.remote.webelement.WebElement (session="b8a3158143a469d27d6821d2b81a648f", element="c3805f76-5f26-4ad2-8f19-e5c3e282413a")>

In [38]:
secciones_menu = menu_lateral.find_elements_by_tag_name("a")

2. Vemos con cuál nos tenemos que quedar

In [39]:
for a in secciones_menu:
    if a.text == "Próximos estrenos":
        a.click()
        break

Accedemos al container central, en el que aparecen los estrenos por semana que queremos ver, exactamente igual que hemos hecho antes

In [40]:
cajon_central = driver.find_elements_by_id("main-wrapper-rdcat")

In [41]:
type(cajon_central)

list

In [43]:
for semana in cajon_central:
    print(semana.find_element_by_tag_name("div").text)
    print(semana.find_element_by_tag_name("div").get_attribute("id"))

21 de mayo de 2021
2021-05-21
28 de mayo de 2021
2021-05-28
3 de junio de 2021
2021-06-03
4 de junio de 2021
2021-06-04
11 de junio de 2021
2021-06-11
16 de junio de 2021
2021-06-16
18 de junio de 2021
2021-06-18
25 de junio de 2021
2021-06-25


In [ ]:
for semana in cajon_central:
    fecha = semana.find_element_by_tag_name("div").get_attribute("id")
    if fecha == "2021-05-21":
        break

Buscamos cómo acceder a las películas

In [46]:
caratula = semana.find_elements_by_class_name("mc-poster")
lista_pelis = []
for peli in caratula:
    lista_pelis.append(peli.find_element_by_tag_name("a").get_attribute("href"))

In [47]:
lista_pelis

['https://www.filmaffinity.com/es/film699806.html',
 'https://www.filmaffinity.com/es/film221888.html',
 'https://www.filmaffinity.com/es/film509996.html',
 'https://www.filmaffinity.com/es/film266841.html',
 'https://www.filmaffinity.com/es/film829002.html',
 'https://www.filmaffinity.com/es/film292437.html']

Una vez tenemos todas las urls vamos a ver qué hacemos con cada una de ellas

In [54]:
# Accedemos a la página de la primera pelicula
driver.get(lista_pelis[0])

In [56]:
driver.back()

Vamos a ver el proceso que deberíamos hacer con cada una de las películas:

1. Sacamos toda la información que nos interesa

In [59]:
# titulo, nota, numero de votos y ficha técnica
titulo = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span").text
nota = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]").text
votos = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span").text
ficha = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]")

2. Creamos una lista a partir de la ficha técnica

In [60]:
# Los nombres estan con tag  = 'dt' y los valores con 'dd'
ficha_claves = []
ficha_valores = []
for name in ficha.find_elements_by_tag_name("dt"):
    ficha_claves.append(name.text)
for value in ficha.find_elements_by_tag_name("dd"):
    ficha_valores.append(value.text)

In [61]:
ficha_valores

['Spiral: From the Book of Saw',
 '2021',
 '93 min.',
 ' Estados Unidos',
 'Darren Lynn Bousman',
 'Josh Stolberg, Pete Goldfinger. Historia: Pete Goldfinger, Chris Rock, Josh Stolberg',
 'Charlie Clouser',
 'Jordan Oram',
 'Chris Rock, Samuel L. Jackson, Max Minghella, Marisol Nichols, Zoie Palmer, Nazneen Contractor, Dan Petronijevic, Morgan David Jones, Edie Inksetter, K.C. Collins, Josh Stolberg, Ali Johnson, Brian Cook, ver 6 más',
 'Serendipity Productions, Twisted Pictures, Burg/Koules Productions, Dahlstar. Distribuidora: Lionsgate',
 'Terror. Thriller | Policíaco. Crimen. Asesinos en serie',
 'Saw',
 'Trabajando a la sombra de un policía veterano (Samuel L. Jackson), el intrépido detective Ezekiel “Zeke” Banks (Chris Rock) y su compañero novato (Max Minghella) toman las riendas de una investigación relacionada con una serie de asesinatos que albergan ciertos paralelismos con el sangriento pasado de la ciudad. A medida que profundiza en el misterio, Zeke descubre que él mismo s

3. Creamos un dataframe con la info

In [68]:
columns = ["titulo", "nota", "votos"]
columns.extend(ficha_claves)
values = [titulo, nota, votos]
values.extend(ficha_valores)
len(values)

16

In [69]:
pd.DataFrame([values], columns= columns)

,titulo,nota,votos,Título original,Año,Duración,País,Dirección,Guion,Música,Fotografía,Reparto,Productora,Género,Grupos,Sinopsis
0,Spiral: Saw,"4,9",39,Spiral: From the Book of Saw,2021,93 min.,Estados Unidos,Darren Lynn Bousman,"Josh Stolberg, Pete Goldfinger. Historia: Pete...",Charlie Clouser,Jordan Oram,"Chris Rock, Samuel L. Jackson, Max Minghella, ...","Serendipity Productions, Twisted Pictures, Bur...",Terror. Thriller | Policíaco. Crimen. Asesinos...,Saw,Trabajando a la sombra de un policía veterano ...


Ahora vamos a crear una función que nos haga todo esto para cada una de las películas:

In [70]:
def sacar_info(driver):
    titulo = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span").text
    try:
        nota = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]").text
        votos = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span").text
    except:
        nota = None
        votos = None

    ficha = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]")
    return titulo, nota, votos, ficha

def sacar_ficha():
    ficha_claves = []
    ficha_valores = []
    for name in ficha.find_elements_by_tag_name("dt"):
        ficha_claves.append(name.text)
    for value in ficha.find_elements_by_tag_name("dd"):
        ficha_valores.append(value.text)
    return ficha_claves, ficha_valores
  
def montar_df(ficha_claves,ficha_valores, titulo, nota,votos):
    columns = ["titulo", "nota", "votos"]
    columns.extend(ficha_claves)
    values = [titulo, nota, votos]
    values.extend(ficha_valores)

    return pd.DataFrame([values], columns = columns)

def nueva_peli(driver):
    titulo,nota,votos,ficha = sacar_info(driver)
    ficha_claves , ficha_valores = sacar_ficha(ficha)
    df_peli = montar_df(ficha_claves, ficha_valores, titulo, nota, votos)


## Modo Dios: moviendonos entre ventanas

Vamos a ver cómo nos podemos mover entre ventanas del navegador

Abrir nueva ventana:

In [77]:
driver.execute_script('window.open("");')

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=90.0.4430.212)


Movernos a otra ventana

In [72]:
driver.switch_to.window(driver.window_handles[0])

Cerrar ventana

In [73]:
#siempre que cierro una ventana, tengo que decirle al driver que se mueva a la siguiente
driver.close()

Una vez cerramos la ventana tenemos que indicarle a qué ventana tiene que ir

Sabiendo cómo podemos movernos por entre las ventanas y sabiendo cómo extraer de cada página toda la información que necesitamos vamos a crear nuestro dataframe:

In [75]:
# para abrir todos los links en lista_pelis
for link in lista_pelis:
    driver.execute_script("window.open('"+link+"');")
    driver.switch_to.window(driver.window_handles[-l])
    driver.get(link)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=90.0.4430.212)


In [76]:
# Creamos un dataframe con todas las pelis que se estrenan la próxima semana:
df_peliculas = pd.DataFrame()
for link in lista_pelis:
    driver.execute_script("window.open('"+link+"');")
    driver.switch_to.window(driver.window_handles[-l])
    nueva_pelicula = nueva_peli(driver)
    df_peliculas = df_peliculas.append(nueva_pelicula)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=90.0.4430.212)


¡Tachán! Ya tenemos un dataframe con todas las películas que se van a estrenar el próximo viernes